In [612]:
import pandas as pd
import re
import os
import numpy as np
import datetime

In [613]:
csv_list = []
LUIGI_LOG_PATH='/usr/local/trustyou/home/andream/nfs/processmining/minilogs'
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv'):
        # log_path = LUIGI_LOG_PATH+'/'+filename
        csv_list.append(filename)
csv_list

['daily.2019-09-04_23-23-01.csv',
 'daily.2019-09-09_01-34-02.csv',
 'daily.2019-09-16_19-44-01.csv',
 'daily.2019-09-20_12-49-01.csv',
 'daily.2019-09-24_05-05-02.csv',
 'daily.2019-09-11_05-40-02.csv',
 'daily.2019-09-13_04-11-01.csv',
 'daily.2019-09-06_20-34-01.csv',
 'daily.2019-09-18_14-08-01.csv',
 'daily.2019-09-15_01-41-01.csv',
 'daily.2019-09-03_03-44-01.csv',
 'daily.2019-09-22_07-40-01.csv',
 'daily.2019-09-01_09-29-01.csv']

In [614]:
filename='daily.2019-09-04_23-23-01.csv'
df = pd.read_csv(LUIGI_LOG_PATH+'/'+filename, index_col=0)

# df['pid'] = df['pid'].astype(str)  
# df['pid'] = df['pid'].str.replace('.0', ' ')
# df['pid'] = df['pid'].replace('nan', np.nan, regex=True)

df = df[['timestamp','task','state']]
df['source_file'] = filename

df['state'][0]='start'
df['task'][0]='AllTasks'

df['state'][len(df)-1]='done'
df['task'][len(df)-1]='AllTasks'

print(len(df))
df.head()

2506


,timestamp,task,state,source_file
0,2019-09-04 23:23:20,AllTasks,start,daily.2019-09-04_23-23-01.csv
1,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=daily...",start,daily.2019-09-04_23-23-01.csv
2,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=regul...",start,daily.2019-09-04_23-23-01.csv
3,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=repro...",start,daily.2019-09-04_23-23-01.csv
4,2019-09-04 23:35:08,"ProxySetupTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep)\n",start,daily.2019-09-04_23-23-01.csv


In [615]:
def get_task(task_call):
    if isinstance(task_call, float):
        return task_call
    return task_call.split('(')[0]

In [616]:
df['task_call'] = df['task']
df['task_name'] = np.where(df['task'].notnull(), df['task'].apply(get_task), df['task'])
df = df[['task_name','state','timestamp','task_call', 'source_file']]

In [595]:
df['start_time'] = np.where(df['state']=='start', df['timestamp'], np.nan)
df['end_time'] = np.where(df['state']=='done', df['timestamp'], np.nan)

In [596]:
start_df = df[df['state']=='start']
print(len(start_df))

end_df = df[df['state']=='done']
print(len(end_df))

1252
1248


In [597]:
all_df = pd.merge(start_df, end_df, on=['task_call', 'source_file'], how='outer')
print(len(all_df))

1252


In [598]:
def get_duration(start_time, end_time):
    start = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    end = datetime.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')
    duration = abs(end - start)
    return duration
# get_duration(ex['timestamp'][10],ex['timestamp'][1])

In [606]:
all_df['start_time'] = all_df['start_time_x']
all_df['end_time'] = all_df['end_time_y']
all_df['duration'] = all_df.apply(lambda row: str(get_duration(row['start_time'],row['end_time'])), axis=1)
all_df['t_duration']= all_df.apply(lambda row: get_duration(row['start_time'],row['end_time']), axis=1)
all_df['task_name'] = all_df['task_name_x']

dur_df = all_df[['task_name','duration','task_call','start_time','end_time', 'source_file']]
print(len(dur_df))
dur_df = dur_df.sort_values(by=['task_call'])
dur_df.head()

1252


,task_name,duration,task_call,start_time,end_time,source_file
943,AlertsTask,0:00:02,"AlertsTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=01, send_to=real)\n",2019-09-05 17:50:00,2019-09-05 17:50:02,daily.2019-09-04_23-23-01.csv
1224,AlertsTask,0:00:01,"AlertsTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=02, send_to=real)\n",2019-09-06 16:19:34,2019-09-06 16:19:35,daily.2019-09-04_23-23-01.csv
0,AllTasks,"1 day, 21:09:45",AllTasks,2019-09-04 23:23:20,2019-09-06 20:33:05,daily.2019-09-04_23-23-01.csv
888,ChunkReviewTimelineTask,0:03:16,"ChunkReviewTimelineTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=01)\n",2019-09-05 16:54:08,2019-09-05 16:57:24,daily.2019-09-04_23-23-01.csv
1200,ChunkReviewTimelineTask,0:05:26,"ChunkReviewTimelineTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=02)\n",2019-09-06 14:33:25,2019-09-06 14:38:51,daily.2019-09-04_23-23-01.csv


In [601]:
#for name, group in df.groupby(['task_call'],as_index=False):
#    print(name)
#    print(group)

In [602]:
pd.options.display.max_colwidth = 100
grouped = df.groupby(['task_call'],as_index=False)
grouped.size()

task_call
AlertsTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=01, send_to=real)\n                                                                                                                                   2
AlertsTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=02, send_to=real)\n                                                                                                                                   2
AllTasks                                                                                                                                                                                                                        2
ChunkReviewTimelineTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=01)\n                                                                                                                                    2
ChunkReviewTimelineTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk